In [ ]:
import pandas
import sklearn.feature_extraction.text
import sklearn.metrics.pairwise

In [ ]:
# load mimic 2 data
annotations = pandas.read_csv('../data/raw/annotations.csv')

In [ ]:
annotations.head()

In [ ]:
# load mimic 3 data
m3_chunks = pandas.read_csv('../data/raw/NOTEEVENTS.csv', chunksize=100000)

In [ ]:
%%time
res = pandas.concat(pandas.merge(left=annotations, right=m3_chunk, how='inner', left_on='subject.id', right_on='SUBJECT_ID') for m3_chunk in m3_chunks)

In [ ]:
res.columns

In [ ]:
res.head()

In [ ]:
res.shape

In [ ]:
cv = sklearn.feature_extraction.text.CountVectorizer()

In [ ]:
%%time
cv.fit(res['text'], res['TEXT'])

In [ ]:
%%time
res['note_similarity'] = res.apply(lambda x: 
    sklearn.metrics.pairwise.cosine_similarity(
        cv.transform([x['text']]), 
        cv.transform([x['TEXT']]))[0,0]
    ,axis=1)

In [ ]:
%%time
reb = res
reb = reb.sort_values(by='note_similarity', ascending=False).drop_duplicates(subset=['subject.id', 'text'], keep='first').sort_index()

In [ ]:
reb = reb.rename(columns={'ROW_ID': 'mimic_iii_row_id'})
reb.head()

In [ ]:
cols = [u'mimic_iii_row_id', u'text', u'Hospital.Admission.ID', u'subject.id', u'chart.time',
       u'cohort', u'Obesity', u'Non.Adherence',
       u'Developmental.Delay.Retardation', u'Advanced.Heart.Disease',
       u'Advanced.Lung.Disease',
       u'Schizophrenia.and.other.Psychiatric.Disorders', u'Alcohol.Abuse',
       u'Other.Substance.Abuse', u'Chronic.Pain.Fibromyalgia',
       u'Chronic.Neurological.Dystrophies', u'Advanced.Cancer', u'Depression',
       u'Dementia', u'Unsure']

In [ ]:
reb = reb.filter(items=cols)
reb.head()

In [ ]:
# sort it
merged = pandas.merge(left=annotations, right=reb, how='inner', on=['Hospital.Admission.ID', 'subject.id', 'chart.time', 'text'], suffixes=['','_22'])
ren = merged.filter(items=cols)

In [ ]:
ren.to_csv('../data/raw/merged_annotations.csv', index=False)